In [2]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from datetime import datetime
import xarray as xr
# from mpl_toolkits.basemap import Basemap

In [9]:
base_dir = "/Users/yusukehatanaka/Desktop/StatisticalDownscaling"
# load all the 16 netcdf files
# ds_air2m = nc.Dataset(f"{base_dir}/air.2m.mon.mean.regridded.nc")
# ds_air1000_500 = nc.Dataset(f"{base_dir}/air.1000-500.mon.mean.nc")
# ds_hgt500 = nc.Dataset(f"{base_dir}/hgt500.mon.mean.nc")
# ds_hgt1000 = nc.Dataset(f"{base_dir}/hgt1000.mon.mean.nc")
# ds_omega500 = nc.Dataset(f"{base_dir}/omega500.mon.mean.nc")
# ds_pottemp_1000_500 = nc.Dataset(f"{base_dir}/pottmp.1000-500.mon.mean.nc")
# ds_pottemp_1000_850 = nc.Dataset(f"{base_dir}/pottmp.1000-850.mon.mean.nc")
# ds_pwtr = nc.Dataset(f"{base_dir}/pwtr.mon.mean.nc")
# ds_u700 = nc.Dataset(f"{base_dir}/shum_x_uwnd.700.mon.mean.nc")
# ds_u925 = nc.Dataset(f"{base_dir}/shum_x_uwnd.925.mon.mean.nc")
# ds_v700 = nc.Dataset(f"{base_dir}/shum_x_vwnd.700.mon.mean.nc")
# ds_v950 = nc.Dataset(f"{base_dir}/shum_x_vwnd.925.mon.mean.nc")
# ds_shum700 = nc.Dataset(f"{base_dir}/shum700.mon.mean.nc")
# ds_shum925 = nc.Dataset(f"{base_dir}/shum925.mon.mean.nc")
# ds_skt = nc.Dataset(f"{base_dir}/skt.mon.mean.regridded.nc")
# ds_slp = nc.Dataset(f"{base_dir}/slp.mon.mean.nc")

ds_air2m = xr.open_dataset(f"{base_dir}/air.2m.mon.mean.regridded.nc")
ds_air1000_500 = xr.open_dataset(f"{base_dir}/air.1000-500.mon.mean.nc")
ds_hgt500 = xr.open_dataset(f"{base_dir}/hgt500.mon.mean.nc")
ds_hgt1000 = xr.open_dataset(f"{base_dir}/hgt1000.mon.mean.nc")
ds_omega500 = xr.open_dataset(f"{base_dir}/omega500.mon.mean.nc")
ds_pottemp_1000_500 = xr.open_dataset(f"{base_dir}/pottmp.1000-500.mon.mean.nc")
ds_pottemp_1000_850 = xr.open_dataset(f"{base_dir}/pottmp.1000-850.mon.mean.nc")
ds_pwtr = xr.open_dataset(f"{base_dir}/pwtr.mon.mean.nc")
ds_u700 = xr.open_dataset(f"{base_dir}/shum_x_uwnd.700.mon.mean.nc")
ds_u925 = xr.open_dataset(f"{base_dir}/shum_x_uwnd.925.mon.mean.nc")
ds_v700 = xr.open_dataset(f"{base_dir}/shum_x_vwnd.700.mon.mean.nc")
ds_v950 = xr.open_dataset(f"{base_dir}/shum_x_vwnd.925.mon.mean.nc")
ds_shum700 = xr.open_dataset(f"{base_dir}/shum700.mon.mean.nc")
ds_shum925 = xr.open_dataset(f"{base_dir}/shum925.mon.mean.nc")
ds_skt = xr.open_dataset(f"{base_dir}/skt.mon.mean.regridded.nc")
ds_slp = xr.open_dataset(f"{base_dir}/slp.mon.mean.nc")

# nc.Dataset(f"{base_dir}/")

In [4]:
df_locations = pd.read_csv("SKNlocations.csv")

In [11]:
# load all the 16 netcdf files
ds_entries = [
    ds_air2m,
    ds_air1000_500,
    ds_hgt500,
    ds_hgt1000,
    ds_omega500,
    ds_pottemp_1000_500,
    ds_pottemp_1000_850,
    ds_pwtr,
    ds_u700,
    ds_u925,
    ds_v700,
    ds_v950,
    ds_shum700,
    ds_shum925,
    ds_skt,
    ds_slp
]

In [12]:
# Make sure for all nectcf files, lat and lon are consistent and that
# lat = np.arange(90, -91, -2.5)
# lon = np.arange(0, 365, 2.5)
for entry in ds_entries:
    test_lat = entry["lat"] == np.arange(90, -91, -2.5)
    test_lon = entry["lon"] == np.arange(0, 360, 2.5)
    assert np.all(test_lat) == True
    assert np.all(test_lon) == True

In [17]:
for entry in ds_entries:
    #print(entry["time"][:].min())
    #print(entry["time"][:].max())
    continue

In [20]:
ds_slp["lon"]

<xarray.DataArray 'lon' (lon: 144)>
array([  0. ,   2.5,   5. ,   7.5,  10. ,  12.5,  15. ,  17.5,  20. ,  22.5,
        25. ,  27.5,  30. ,  32.5,  35. ,  37.5,  40. ,  42.5,  45. ,  47.5,
        50. ,  52.5,  55. ,  57.5,  60. ,  62.5,  65. ,  67.5,  70. ,  72.5,
        75. ,  77.5,  80. ,  82.5,  85. ,  87.5,  90. ,  92.5,  95. ,  97.5,
       100. , 102.5, 105. , 107.5, 110. , 112.5, 115. , 117.5, 120. , 122.5,
       125. , 127.5, 130. , 132.5, 135. , 137.5, 140. , 142.5, 145. , 147.5,
       150. , 152.5, 155. , 157.5, 160. , 162.5, 165. , 167.5, 170. , 172.5,
       175. , 177.5, 180. , 182.5, 185. , 187.5, 190. , 192.5, 195. , 197.5,
       200. , 202.5, 205. , 207.5, 210. , 212.5, 215. , 217.5, 220. , 222.5,
       225. , 227.5, 230. , 232.5, 235. , 237.5, 240. , 242.5, 245. , 247.5,
       250. , 252.5, 255. , 257.5, 260. , 262.5, 265. , 267.5, 270. , 272.5,
       275. , 277.5, 280. , 282.5, 285. , 287.5, 290. , 292.5, 295. , 297.5,
       300. , 302.5, 305. , 307.5, 310. , 312.5, 315. , 317.5, 320. , 322.5,
       325. , 327.5, 330. , 332.5, 335. , 337.5, 340. , 342.5, 345. , 347.5,
       350. , 352.5, 355. , 357.5], dtype=float32)
Coordinates:
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Attributes:
    units:          degrees_east
    long_name:      Longitude
    actual_range:   [  0.  357.5]
    standard_name:  longitude
    axis:           X

In [31]:
ds = xr.open_dataset(f"{base_dir}/air.2m.mon.mean.regridded.nc")

In [31]:
df = ds.to_dataframe().reorder_levels(["time", "lat", "lon"])

In [ ]:
# 1. make a table for all the stations where the ds coordinates are the closest
# Given a 2D table (X, Y), and target vector of (x, y), return the point in (X, Y) that is closest to (x, y)

def month_to_value(year, month, ds, key, x, y):
    # Given dataset and key, first checks if the (year, month) information is included,
    # and if it does, then extract the value at (x, y).
    # Specification: (x, y) has to exist in xarray ds[key][:]

In [33]:
ds["air"].loc['1948-03-01'].loc["0"]

<xarray.DataArray 'air' (lon: 144)>
array([300.44983, 300.43005, 300.35046, 300.93707, 298.0597 , 294.9086 ,
       295.20288, 296.11285, 296.20782, 295.81503, 295.4579 , 293.11163,
       291.64917, 293.40128, 292.90967, 294.72552, 301.25766, 302.34586,
       300.121  , 300.08304, 299.71265, 299.76614, 299.8558 , 299.86023,
       299.9932 , 299.99994, 299.99554, 300.1292 , 300.31747, 300.22952,
       300.16678, 300.0008 , 300.0677 , 300.11584, 300.0245 , 300.09225,
       299.88373, 300.1759 , 300.20752, 299.38855, 297.1206 , 296.7868 ,
       299.6284 , 299.76303, 297.60568, 295.52368, 295.35156, 297.34445,
       297.7931 , 298.13165, 299.39508, 299.59616, 299.4044 , 298.52597,
       299.25354, 300.11166, 300.30188, 300.5868 , 300.5408 , 300.3305 ,
       300.33868, 300.39   , 300.23688, 300.0473 , 299.91174, 299.86185,
       299.87906, 299.8025 , 299.90366, 299.9419 , 299.77655, 299.6502 ,
       299.54242, 299.51126, 299.4362 , 299.40225, 299.38245, 299.43124,
       299.52823, 299.29453, 299.24548, 299.17325, 299.0244 , 299.2658 ,
       299.12695, 298.91336, 298.96118, 298.995  , 299.06622, 298.80032,
       298.7663 , 298.79453, 298.6128 , 298.63776, 298.46594, 298.47705,
       298.69327, 298.6584 , 298.64258, 298.7905 , 298.80704, 298.9296 ,
       299.0274 , 299.0941 , 299.30365, 299.214  , 299.3036 , 298.93652,
       299.07257, 299.22684, 298.7688 , 299.05518, 294.5875 , 291.70404,
       295.2137 , 296.35065, 296.3712 , 296.37952, 296.40756, 296.6674 ,
       296.06747, 296.4493 , 296.2044 , 296.26562, 297.52042, 298.2881 ,
       299.41595, 299.54248, 299.53012, 299.46454, 299.2814 , 299.47235,
       299.51242, 299.50317, 299.50922, 299.68967, 299.51297, 299.50223,
       299.82147, 299.88733, 299.89   , 299.70953, 299.87897, 300.0692 ],
      dtype=float32)
Coordinates:
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    lat      float32 0.0
    time     datetime64[ns] 1948-03-01
Attributes:
    long_name:     Monthly Mean of Air Temperature
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    level_desc:    2 m
    statistic:     Mean
    parent_stat:   Individual Obs
    dataset:       NCEP Reanalysis Derived Products
    actual_range:  [187.4545  314.80725]

In [ ]:
["air", "air", "hgt", "hgt", ]

In [73]:
ds_entries[3]

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    description:  Data from NCEP initialized reanalysis (4x/day).  These are interpolated to pressure surfaces from model (sigma) surfaces.
    platform: Model
    Conventions: COARDS
    NCO: 4.2.6
    history: Sun Aug 26 20:12:39 2018: ncks -O -d level,1000.000000 -d lat,-90.000000,90.000000 -d lon,0.000000,357.500000 -d time,0,839 /Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc /Public/www/X98.151.202.176.237.20.12.38.nc
Created by NOAA-CIRES Climate Diagnostics Center (SAC) from the NCEP
reanalysis data set on 07/07/97 by calc.mon.mean.year.f using
/Datasets/nmc.reanalysis.derived/pressure/hgt.mon.mean.nc
from /Datasets/nmc.reanalysis/pressure/hgt.79.nc to hgt.95.nc
Converted to chunked, deflated non-packed NetCDF4 2014/09
    title: monthly mean hgt from the NCEP Reanalysis
    References: http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis.derived.html
    dataset_t

In [49]:
def hours_to_datetime(hours):
    # takes in the number of hours since 1/1/1800 and returns datetime object
    return datetime(1800, 1, 1) + timedelta(hours = hours)

In [60]:
for entry in ds_entries:
    # print(entry["time"][:].shape)
    # print(entry["time"][:][0])
    # print(entry["time"][:][-1])
    # print(entry["time"][:][0])
    hours = entry["time"][:][-1]
    print(hours_to_datetime(hours))    

2019-08-01 00:00:00
2019-08-01 00:00:00
2017-12-01 00:00:00
2017-12-01 00:00:00
2018-04-01 00:00:00
2019-08-01 00:00:00
2019-08-01 00:00:00
2018-04-01 00:00:00
2019-08-01 00:00:00
2019-08-01 00:00:00
2019-08-01 00:00:00
2019-08-01 00:00:00
2017-12-01 00:00:00
2018-04-01 00:00:00
2019-07-01 00:00:00
2019-08-01 00:00:00


In [61]:
print(ds_entries[0]["air"][0])

[[234.86572 234.86572 234.86572 ... 234.86572 234.86572 234.86572]
 [235.7996  235.8135  235.79033 ... 235.81003 235.8116  235.80731]
 [238.04106 238.373   238.67155 ... 236.87514 237.26465 237.66283]
 ...
 [256.27634 256.5619  256.8327  ... 255.57182 255.76965 256.01227]
 [256.82468 256.86514 256.9081  ... 256.74506 256.75967 256.7929 ]
 [256.85266 256.85266 256.85266 ... 256.85266 256.85266 256.85266]]


In [68]:
for time, value in zip(ds_entries[0]["time"], ds_entries[0]["air"]):
    print(hours_to_datetime(int(time)).year, value.shape)

1948 (73, 144)
1948 (73, 144)
1948 (73, 144)
1948 (73, 144)
1948 (73, 144)
1948 (73, 144)
1948 (73, 144)
1948 (73, 144)
1948 (73, 144)
1948 (73, 144)
1948 (73, 144)
1948 (73, 144)
1949 (73, 144)
1949 (73, 144)
1949 (73, 144)
1949 (73, 144)
1949 (73, 144)
1949 (73, 144)
1949 (73, 144)
1949 (73, 144)
1949 (73, 144)
1949 (73, 144)
1949 (73, 144)
1949 (73, 144)
1950 (73, 144)
1950 (73, 144)
1950 (73, 144)
1950 (73, 144)
1950 (73, 144)
1950 (73, 144)
1950 (73, 144)
1950 (73, 144)
1950 (73, 144)
1950 (73, 144)
1950 (73, 144)
1950 (73, 144)
1951 (73, 144)
1951 (73, 144)
1951 (73, 144)
1951 (73, 144)
1951 (73, 144)
1951 (73, 144)
1951 (73, 144)
1951 (73, 144)
1951 (73, 144)
1951 (73, 144)
1951 (73, 144)
1951 (73, 144)
1952 (73, 144)
1952 (73, 144)
1952 (73, 144)
1952 (73, 144)
1952 (73, 144)
1952 (73, 144)
1952 (73, 144)
1952 (73, 144)
1952 (73, 144)
1952 (73, 144)
1952 (73, 144)
1952 (73, 144)
1953 (73, 144)
1953 (73, 144)
1953 (73, 144)
1953 (73, 144)
1953 (73, 144)
1953 (73, 144)
1953 (73, 